In [1]:
import numpy as np
import timor
import argparse
import pygad
import os
from util import *
from timor.Module import *
from timor.utilities.visualization import animation
# import matplotlib.pyplot as plt
# import itertools
# from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from timor import ModuleAssembly, ModulesDB
from timor.Bodies import Body, Connector, Gender
from timor.configuration_search.GA import GA
from timor.utilities.visualization import MeshcatVisualizer, clear_visualizer
from timor.utilities.dtypes import Lexicographic
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor.Joints import Joint
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere, Mesh

from timor_ga import *
from reachability import Reachability
from generate_module import create_i_links, create_eef, create_revolute_joint, base

2025-03-10 22:46:05,576 Timor INFO Loading custom configurations from /home/mscsim/.config/timor.config
2025-03-10 22:46:05,582 Timor INFO Getting robot modrob-gen2.
2025-03-10 22:46:05,583 Timor WARNING Could not fetch robot modrob-gen2 due to [Errno 17] File exists: '/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/cache/robots/modrob-gen2'.


In [2]:
how_many_times_to_split_angle_range = 30
world_resolution = 0.01
world_dimension = [1.00, 1.00, 1.00]
num_threads = 5
our_hyperparameters = {
    'population_size': 30,
    'num_generations': 100,
    'num_genes': 9,
    'save_solutions_dir': None
}

In [2]:
#r_4310_base = create_revolute_joint("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
r_4310_base = base("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
r_4305_joint = create_revolute_joint("assets/Assem_4305_JOINT/Assem_4305_JOINT/urdf/Assem_4305_JOINT.urdf")
r_4310_joint = create_revolute_joint("assets/Assem_4310_JOINT/Assem_4310_JOINT/urdf/Assem_4310_JOINT.urdf")

# Links

baseto4310_links = create_i_links(rod_name="baseto4310")
r4310to4305_links = create_i_links(rod_name="r4310to4305")
r4310to4310_links = create_i_links(rod_name="r4310to4310")

eef = create_eef()

# Create database
db = ModulesDB()
db.add(r_4310_base)
db.add(r_4310_joint)
db.add(r_4305_joint)
db = db.union(baseto4310_links)
db = db.union(r4310to4305_links)
db = db.union(r4310to4310_links)

db = db.union(eef)
viz = db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:
#modules = ('base_rev_joint','baseto4310-0.3','motor4310_rev_joint', 'r4310to4310-0.3', 'motor4310_rev_joint', 'r4310to4305-0.15', 'motor4305_rev_joint', 'eef')
#modules = ('base_rev_joint', 'baseto4310-0.45', 'motor4305_rev_joint', 'r4310to4305-0.45', 'motor4305_rev_joint', 'eef')
#modules = ('base_rev_joint', 'baseto4310-0.45', 'motor4305_rev_joint', 'r4310to4305-0.3', 'motor4310_rev_joint', 'r4310to4305-0.15', 'r4310to4310-0.15', 'motor4310_rev_joint', 'eef')
#modules = ('base_rev_joint', 'baseto4310-0.3', 'r4310to4310-0.15', 'baseto4310-0.3', 'motor4305_rev_joint', 'r4310to4305-0.15', 'motor4310_rev_joint', 'r4310to4305-0.3', 'baseto4310-0.3', 'motor4305_rev_joint', 'r4310to4310-0.15', 'baseto4310-0.15', 'motor4305_rev_joint', 'r4310to4305-0.15', 'r4310to4310-0.45', 'r4310to4305-0.15', 'motor4310_rev_joint', 'r4310to4305-0.15', 'eef')
modules = ('base_rev_joint', 'baseto4310-0.15', 'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.3', 'r4310to4310-0.15', 'baseto4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.3', 'r4310to4310-0.45', 'baseto4310-0.3', 'motor4310_rev_joint', 'r4310to4310-0.45', 'baseto4310-0.15', 'baseto4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.3', 'baseto4310-0.3', 'eef')
B = ModuleAssembly.from_serial_modules(db, modules)
print(is_valid_module_order(B))
print(B.original_module_ids)
#print(B.nJoints)
robot = B.to_pin_robot()
print(robot.has_self_collision())
print(robot.data)
print(robot.joints)
viz = robot.visualize()

('baseto4310-0.15', 'r4310to4310-0.15')
('base_rev_joint', 'baseto4310-0.15', 'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.3', 'r4310to4310-0.15', 'baseto4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.3', 'r4310to4310-0.45', 'baseto4310-0.3', 'motor4310_rev_joint', 'r4310to4310-0.45', 'baseto4310-0.15', 'baseto4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.3', 'baseto4310-0.3', 'eef')
False
['base_rev_joint.base_rev_joint', 'motor4310_rev_joint.motor4310_rev_joint', 'motor4310_rev_joint-1.motor4310_rev_joint', 'motor4310_rev_joint-2.motor4310_rev_joint', 'motor4310_rev_joint-3.motor4310_rev_joint']
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [5]:
modules = ('base_rev_joint', 'baseto4310-0.15', 'motor4310_rev_joint', 
           'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.15', 
           'motor4310_rev_joint', 'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4305-0.15', 'motor4305_rev_joint')
#modules = ('base_rev_joint', 'baseto4310-0.3', 'motor4310_rev_joint', 'r4310to4310-0.3', 'motor4310_rev_joint', 'r4310to4305-0.3', 'motor4305_rev_joint', 'eef')
B = ModuleAssembly.from_serial_modules(db, modules)
print(is_valid_module_order(B))
robot = B.to_pin_robot()

B.to_urdf(write_to=Path("robot6DOF.urdf"))
print("DOF of the robot:", robot.dof)

# for frame in robot.model.frames:
#     print(frame.name)
print(robot.parents.keys())
print(robot.joints)
viz = robot.visualize()


True
DOF of the robot: 6
dict_keys(['base_rev_joint.base_rev_joint', 'motor4310_rev_joint.motor4310_rev_joint', 'motor4310_rev_joint-1.motor4310_rev_joint', 'motor4310_rev_joint-2.motor4310_rev_joint', 'motor4310_rev_joint-3.motor4310_rev_joint', 'motor4305_rev_joint.motor4305_rev_joint'])
['base_rev_joint.base_rev_joint', 'motor4310_rev_joint.motor4310_rev_joint', 'motor4310_rev_joint-1.motor4310_rev_joint', 'motor4310_rev_joint-2.motor4310_rev_joint', 'motor4310_rev_joint-3.motor4310_rev_joint', 'motor4305_rev_joint.motor4305_rev_joint']
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [6]:
from timor.utilities.visualization import animation
q0 = np.zeros((robot.dof,))
print(robot.dof)
q1 = robot.random_configuration()
q2 = np.array([np.pi, -np.pi/2, np.pi/2, np.pi/2])
print(q1)
robot.update_configuration(q1)
print(robot.has_self_collision())
viz = robot.visualize()
#print(q1)

# trajectory = np.linspace(q0, q1)
# print(trajectory)
# animation(robot, trajectory, dt=.1)

6
[-2.90001973 -0.95038434  0.38799116 -0.45670906 -1.64027593 -0.93148132]
False
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7031/static/


In [7]:
from reachability import Reachability
from timor_ga import *

# modules = ('base_rev_joint', 'baseto4310-0.15', 'motor4310_rev_joint', 
#            'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.15', 
#            'motor4310_rev_joint', 'eef')
# modules = ('base_rev_joint', 'baseto4310-0.15', 'motor4310_rev_joint', 
#            'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.15', 
#            'motor4310_rev_joint', 'r4310to4305-0.15', 'motor4310_rev_joint', 'r4310to4305-0.15', 'motor4305_rev_joint')
# C = ModuleAssembly.from_serial_modules(db, modules)

# robot = C.to_pin_robot()

how_many_times_to_split_angle_range = 30
world_resolution = 0.01
world_dimension = [1.00, 1.00, 1.00]
num_threads = 5

reach = Reachability(robot)
reach.reachability_random_sample(num_samples = 100000)
valid_poses = reach.reachability_random_sample(num_samples = 100000)
print("Valid poses:" , valid_poses)
reachable, manipulability =  zip(*valid_poses)
reachable = np.array([list(pt) for pt in reachable])
reachability_score = reach.find_reachibility_percentage(valid_pose=reachable)
print(reachability_score)


Valid poses: [((0.1, -0.24, 0.44), 0.0030524887579886535), ((0.12, 0.01, 0.15), 0.0015861596801416745), ((-0.12, 0.16, 0.18), 0.00035203734890061936), ((-0.16, -0.14, 0.26), 0.0001886254206132369), ((0.21, 0.26, 0.47), 0.01316378080962349), ((-0.32, -0.13, 0.42), 0.007980899938007139), ((-0.04, 0.35, 0.55), 0.001219157022973583), ((0.32, 0.15, 0.03), 0.0024936894967875437), ((0.23, -0.21, -0.04), 0.004951365035778797), ((-0.01, 0.11, 0.11), 0.0028923880098198937), ((-0.19, -0.14, 0.43), 0.0023527261262529637), ((-0.32, 0.23, 0.45), 0.00349385884973243), ((-0.26, 0.31, 0.4), 0.006950817228365578), ((0.24, -0.17, 0.57), 0.003156891736321675), ((-0.09, 0.31, 0.45), 0.014463919897475757), ((0.33, 0.16, 0.45), 0.007744173989382593), ((-0.49, -0.14, 0.08), 0.000318722127018383), ((0.25, -0.45, 0.45), 0.002381216412906667), ((0.11, -0.01, 0.43), 0.0027567690490616623), ((-0.36, -0.2, 0.41), 0.011028136678198987), ((0.21, -0.11, 0.33), 0.012505564610901297), ((0.24, 0.34, 0.13), 0.001067723243

In [9]:
print(valid_poses[0])
print(len(valid_poses)/100000)
print(reachability_score)

((0.1, -0.24, 0.44), 0.0030524887579886535)
0.70704
77.2
